# Bayesian Gaussian CP decomposition

**Author**: Xinyu Chen [[**GitHub homepage**](https://github.com/xinychen)]

**Download**: This Jupyter notebook is at our GitHub repository. If you want to evaluate the code, please download the notebook from the [**transdim**](https://github.com/xinychen/transdim/blob/master/large-imputer/BGCP.ipynb) repository.

This notebook shows how to implement the Bayesian Gaussian CP decomposition (BGCP) model on some real-world data sets. In the following, we will discuss:

- What the Bayesian Gaussian CP decomposition is.

- How to implement BGCP mainly using Python `numpy` with high efficiency.

- How to make imputation on some real-world spatiotemporal datasets.

To overcome the problem of missing values within multivariate time series data, this model takes into account low-rank tensor structure by folding data along day dimension. For an in-depth discussion of BGCP, please see [1].

<div class="alert alert-block alert-info">
<font color="black">
<b>[1]</b> Xinyu Chen, Zhaocheng He, Lijun Sun (2019). <b>A Bayesian tensor decomposition approach for spatiotemporal traffic data imputation</b>. Transportation Research Part C: Emerging Technologies, 98: 73-84. <a href="https://doi.org/10.1016/j.trc.2018.11.003" title="PDF"><b>[PDF]</b></a> 
</font>
</div>

We start by importing the necessary dependencies. We will make use of `numpy` and `scipy`.

In [1]:
import numpy as np
from numpy.random import multivariate_normal as mvnrnd
from scipy.stats import wishart
from numpy.random import normal as normrnd
from scipy.linalg import khatri_rao as kr_prod
from numpy.linalg import inv as inv
from numpy.linalg import solve as solve
from numpy.linalg import cholesky as cholesky_lower
from scipy.linalg import cholesky as cholesky_upper
from scipy.linalg import solve_triangular as solve_ut

In [2]:
def mvnrnd_pre(mu, Lambda):
    src = normrnd(size = (mu.shape[0],))
    return solve_ut(cholesky_upper(Lambda, overwrite_a = True, check_finite = False), 
                    src, lower = False, check_finite = False, overwrite_b = True) + mu

### CP decomposition

#### CP Combination (`cp_combine`)

- **Definition**:

The CP decomposition factorizes a tensor into a sum of outer products of vectors. For example, for a third-order tensor $\mathcal{Y}\in\mathbb{R}^{m\times n\times f}$, the CP decomposition can be written as

$$\hat{\mathcal{Y}}=\sum_{s=1}^{r}\boldsymbol{u}_{s}\circ\boldsymbol{v}_{s}\circ\boldsymbol{x}_{s},$$
or element-wise,

$$\hat{y}_{ijt}=\sum_{s=1}^{r}u_{is}v_{js}x_{ts},\forall (i,j,t),$$
where vectors $\boldsymbol{u}_{s}\in\mathbb{R}^{m},\boldsymbol{v}_{s}\in\mathbb{R}^{n},\boldsymbol{x}_{s}\in\mathbb{R}^{f}$ are columns of factor matrices $U\in\mathbb{R}^{m\times r},V\in\mathbb{R}^{n\times r},X\in\mathbb{R}^{f\times r}$, respectively. The symbol $\circ$ denotes vector outer product.

- **Example**:

Given matrices $U=\left[ \begin{array}{cc} 1 & 2 \\ 3 & 4 \\ \end{array} \right]\in\mathbb{R}^{2\times 2}$, $V=\left[ \begin{array}{cc} 1 & 2 \\ 3 & 4 \\ 5 & 6 \\ \end{array} \right]\in\mathbb{R}^{3\times 2}$ and $X=\left[ \begin{array}{cc} 1 & 5 \\ 2 & 6 \\ 3 & 7 \\ 4 & 8 \\ \end{array} \right]\in\mathbb{R}^{4\times 2}$, then if $\hat{\mathcal{Y}}=\sum_{s=1}^{r}\boldsymbol{u}_{s}\circ\boldsymbol{v}_{s}\circ\boldsymbol{x}_{s}$, then, we have

$$\hat{Y}_1=\hat{\mathcal{Y}}(:,:,1)=\left[ \begin{array}{ccc} 31 & 42 & 65 \\ 63 & 86 & 135 \\ \end{array} \right],$$
$$\hat{Y}_2=\hat{\mathcal{Y}}(:,:,2)=\left[ \begin{array}{ccc} 38 & 52 & 82 \\ 78 & 108 & 174 \\ \end{array} \right],$$
$$\hat{Y}_3=\hat{\mathcal{Y}}(:,:,3)=\left[ \begin{array}{ccc} 45 & 62 & 99 \\ 93 & 130 & 213 \\ \end{array} \right],$$
$$\hat{Y}_4=\hat{\mathcal{Y}}(:,:,4)=\left[ \begin{array}{ccc} 52 & 72 & 116 \\ 108 & 152 & 252 \\ \end{array} \right].$$

In [3]:
def cp_combine(var):
    return np.einsum('is, js, ts -> ijt', var[0], var[1], var[2])

In [4]:
factor = [np.array([[1, 2], [3, 4]]), np.array([[1, 3], [2, 4], [5, 6]]), 
          np.array([[1, 5], [2, 6], [3, 7], [4, 8]])]
print(cp_combine(factor))
print()
print('tensor size:')
print(cp_combine(factor).shape)

[[[ 31  38  45  52]
  [ 42  52  62  72]
  [ 65  82  99 116]]

 [[ 63  78  93 108]
  [ 86 108 130 152]
  [135 174 213 252]]]

tensor size:
(2, 3, 4)


### Tensor Unfolding (`ten2mat`)

Using numpy reshape to perform 3rd rank tensor unfold operation. [[**link**](https://stackoverflow.com/questions/49970141/using-numpy-reshape-to-perform-3rd-rank-tensor-unfold-operation)]

In [5]:
def ten2mat(tensor, mode):
    return np.reshape(np.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1), order = 'F')

In [6]:
def mat2ten(mat, dim, mode):
    index = list()
    index.append(mode)
    for i in range(dim.shape[0]):
        if i != mode:
            index.append(i)
    return np.moveaxis(np.reshape(mat, list(dim[index]), order = 'F'), 0, mode)

### Computing Covariance Matrix (`cov_mat`)

For any matrix $X\in\mathbb{R}^{m\times n}$, `cov_mat` can return a $n\times n$ covariance matrix for special use in the following.

In [7]:
def cov_mat(mat, mat_bar):
    mat = mat - mat_bar
    return mat.T @ mat

## Bayesian Gaussian CP decomposition (BGCP)

### Model Description

#### Gaussian assumption

Given a matrix $\mathcal{Y}\in\mathbb{R}^{m\times n\times f}$ which suffers from missing values, then the factorization can be applied to reconstruct the missing values within $\mathcal{Y}$ by

$$y_{ijt}\sim\mathcal{N}\left(\sum_{s=1}^{r}u_{is} v_{js} x_{ts},\tau^{-1}\right),\forall (i,j,t),$$
where vectors $\boldsymbol{u}_{s}\in\mathbb{R}^{m},\boldsymbol{v}_{s}\in\mathbb{R}^{n},\boldsymbol{x}_{s}\in\mathbb{R}^{f}$ are columns of latent factor matrices, and $u_{is},v_{js},x_{ts}$ are their elements. The precision term $\tau$ is an inverse of Gaussian variance.

#### Bayesian framework

Based on the Gaussian assumption over tensor elements $y_{ijt},(i,j,t)\in\Omega$ (where $\Omega$ is a index set indicating observed tensor elements), the conjugate priors of model parameters (i.e., latent factors and precision term) and hyperparameters are given as

$$\boldsymbol{u}_{i}\sim\mathcal{N}\left(\boldsymbol{\mu}_{u},\Lambda_{u}^{-1}\right),\forall i,$$
$$\boldsymbol{v}_{j}\sim\mathcal{N}\left(\boldsymbol{\mu}_{v},\Lambda_{v}^{-1}\right),\forall j,$$
$$\boldsymbol{x}_{t}\sim\mathcal{N}\left(\boldsymbol{\mu}_{x},\Lambda_{x}^{-1}\right),\forall t,$$
$$\tau\sim\text{Gamma}\left(a_0,b_0\right),$$
$$\boldsymbol{\mu}_{u}\sim\mathcal{N}\left(\boldsymbol{\mu}_0,\left(\beta_0\Lambda_u\right)^{-1}\right),\Lambda_u\sim\mathcal{W}\left(W_0,\nu_0\right),$$
$$\boldsymbol{\mu}_{v}\sim\mathcal{N}\left(\boldsymbol{\mu}_0,\left(\beta_0\Lambda_v\right)^{-1}\right),\Lambda_v\sim\mathcal{W}\left(W_0,\nu_0\right),$$
$$\boldsymbol{\mu}_{x}\sim\mathcal{N}\left(\boldsymbol{\mu}_0,\left(\beta_0\Lambda_x\right)^{-1}\right),\Lambda_x\sim\mathcal{W}\left(W_0,\nu_0\right).$$


### Posterior Inference

In the following, we will apply Gibbs sampling to implement our Bayesian inference for the matrix factorization task.

#### - Sampling latent factors $\boldsymbol{u}_{i},i\in\left\{1,2,...,m\right\}$

Draw $\boldsymbol{u}_{i}\sim\mathcal{N}\left(\boldsymbol{\mu}_i^{*},(\Lambda_{i}^{*})^{-1}\right)$ with following parameters:

$$\boldsymbol{\mu}_{i}^{*}=\left(\Lambda_{i}^{*}\right)^{-1}\left\{\tau\sum_{j,t:(i,j,t)\in\Omega}y_{ijt}\left(\boldsymbol{v}_{j}\circledast\boldsymbol{x}_{t}\right)+\Lambda_u\boldsymbol{\mu}_u\right\},$$

$$\Lambda_{i}^{*}=\tau\sum_{j,t:(i,j,t)\in\Omega}\left(\boldsymbol{v}_{j}\circledast\boldsymbol{x}_{t}\right)\left(\boldsymbol{v}_{j}\circledast\boldsymbol{x}_{t}\right)^{T}+\Lambda_u.$$


#### - Sampling latent factors $\boldsymbol{v}_{j},j\in\left\{1,2,...,n\right\}$

Draw $\boldsymbol{v}_{j}\sim\mathcal{N}\left(\boldsymbol{\mu}_j^{*},(\Lambda_{j}^{*})^{-1}\right)$ with following parameters:

$$\boldsymbol{\mu}_{j}^{*}=\left(\Lambda_{j}^{*}\right)^{-1}\left\{\tau\sum_{i,t:(i,j,t)\in\Omega}y_{ijt}\left(\boldsymbol{u}_{i}\circledast\boldsymbol{x}_{t}\right)+\Lambda_v\boldsymbol{\mu}_v\right\}$$

$$\Lambda_{j}^{*}=\tau\sum_{i,t:(i,j,t)\in\Omega}\left(\boldsymbol{u}_{i}\circledast\boldsymbol{x}_{t}\right)\left(\boldsymbol{u}_{i}\circledast\boldsymbol{x}_{t}\right)^{T}+\Lambda_v.$$


#### - Sampling latent factors $\boldsymbol{x}_{t},t\in\left\{1,2,...,f\right\}$

Draw $\boldsymbol{x}_{t}\sim\mathcal{N}\left(\boldsymbol{\mu}_t^{*},(\Lambda_{t}^{*})^{-1}\right)$ with following parameters:

$$\boldsymbol{\mu}_{t}^{*}=\left(\Lambda_{t}^{*}\right)^{-1}\left\{\tau\sum_{i,j:(i,j,t)\in\Omega}y_{ijt}\left(\boldsymbol{u}_{i}\circledast\boldsymbol{v}_{j}\right)+\Lambda_x\boldsymbol{\mu}_x\right\}$$

$$\Lambda_{t}^{*}=\tau\sum_{i,j:(i,j,t)\in\Omega}\left(\boldsymbol{u}_{i}\circledast\boldsymbol{v}_{j}\right)\left(\boldsymbol{u}_{i}\circledast\boldsymbol{v}_{j}\right)^{T}+\Lambda_x.$$


#### - Sampling precision term $\tau$

Draw $\tau\in\text{Gamma}\left(a^{*},b^{*}\right)$ with following parameters:

$$a^{*}=a_0+\frac{1}{2}|\Omega|,~b^{*}=b_0+\frac{1}{2}\sum_{(i,j,t)\in\Omega}\left(y_{ijt}-\sum_{s=1}^{r}u_{is}v_{js}x_{ts}\right)^2.$$


#### - Sampling hyperparameters $\left(\boldsymbol{\mu}_{u},\Lambda_{u}\right)$

Draw

- $\Lambda_{u}\sim\mathcal{W}\left(W_u^{*},\nu_u^{*}\right)$
- $\boldsymbol{\mu}_{u}\sim\mathcal{N}\left(\boldsymbol{\mu}_{u}^{*},\left(\beta_u^{*}\Lambda_u\right)^{-1}\right)$

with following parameters:

$$\boldsymbol{\mu}_{u}^{*}=\frac{m\boldsymbol{\bar{u}}+\beta_0\boldsymbol{\mu}_0}{m+\beta_0},~\beta_u^{*}=m+\beta_0,~\nu_u^{*}=m+\nu_0,$$
$$\left(W_u^{*}\right)^{-1}=W_0^{-1}+mS_u+\frac{m\beta_0}{m+\beta_0}\left(\boldsymbol{\bar{u}}-\boldsymbol{\mu}_0\right)\left(\boldsymbol{\bar{u}}-\boldsymbol{\mu}_0\right)^T,$$
where $\boldsymbol{\bar{u}}=\sum_{i=1}^{m}\boldsymbol{u}_{i},~S_u=\frac{1}{m}\sum_{i=1}^{m}\left(\boldsymbol{u}_{i}-\boldsymbol{\bar{u}}\right)\left(\boldsymbol{u}_{i}-\boldsymbol{\bar{u}}\right)^T$.


#### - Sampling hyperparameters $\left(\boldsymbol{\mu}_{v},\Lambda_{v}\right)$

Draw

- $\Lambda_{v}\sim\mathcal{W}\left(W_v^{*},\nu_v^{*}\right)$
- $\boldsymbol{\mu}_{v}\sim\mathcal{N}\left(\boldsymbol{\mu}_{v}^{*},\left(\beta_v^{*}\Lambda_v\right)^{-1}\right)$

with following parameters:

$$\boldsymbol{\mu}_{v}^{*}=\frac{n\boldsymbol{\bar{v}}+\beta_0\boldsymbol{\mu}_0}{n+\beta_0},~\beta_v^{*}=n+\beta_0,~\nu_v^{*}=n+\nu_0,$$
$$\left(W_v^{*}\right)^{-1}=W_0^{-1}+nS_v+\frac{n\beta_0}{n+\beta_0}\left(\boldsymbol{\bar{v}}-\boldsymbol{\mu}_0\right)\left(\boldsymbol{\bar{v}}-\boldsymbol{\mu}_0\right)^T,$$
where $\boldsymbol{\bar{v}}=\sum_{j=1}^{n}\boldsymbol{v}_{j},~S_v=\frac{1}{n}\sum_{j=1}^{n}\left(\boldsymbol{v}_{j}-\boldsymbol{\bar{v}}\right)\left(\boldsymbol{v}_{j}-\boldsymbol{\bar{v}}\right)^T$.


#### - Sampling hyperparameters $\left(\boldsymbol{\mu}_{x},\Lambda_{x}\right)$

Draw

- $\Lambda_{x}\sim\mathcal{W}\left(W_x^{*},\nu_x^{*}\right)$
- $\boldsymbol{\mu}_{x}\sim\mathcal{N}\left(\boldsymbol{\mu}_{x}^{*},\left(\beta_x^{*}\Lambda_x\right)^{-1}\right)$

with following parameters:

$$\boldsymbol{\mu}_{x}^{*}=\frac{f\boldsymbol{\bar{x}}+\beta_0\boldsymbol{\mu}_0}{f+\beta_0},~\beta_x^{*}=f+\beta_0,~\nu_x^{*}=f+\nu_0,$$
$$\left(W_x^{*}\right)^{-1}=W_0^{-1}+fS_x+\frac{f\beta_0}{f+\beta_0}\left(\boldsymbol{\bar{x}}-\boldsymbol{\mu}_0\right)\left(\boldsymbol{\bar{x}}-\boldsymbol{\mu}_0\right)^T,$$
where $\boldsymbol{\bar{x}}=\sum_{t=1}^{f}\boldsymbol{x}_{t},~S_x=\frac{1}{f}\sum_{t=1}^{f}\left(\boldsymbol{x}_{t}-\boldsymbol{\bar{x}}\right)\left(\boldsymbol{x}_{t}-\boldsymbol{\bar{x}}\right)^T$.

In [8]:
def sample_factor(tau_sparse_tensor, tau_ind, factor, k, beta0 = 1):
    dim, rank = factor[k].shape
    dim = factor[k].shape[0]
    factor_bar = np.mean(factor[k], axis = 0)
    temp = dim / (dim + beta0)
    var_mu_hyper = temp * factor_bar
    var_W_hyper = inv(np.eye(rank) + cov_mat(factor[k], factor_bar) + temp * beta0 * np.outer(factor_bar, factor_bar))
    var_Lambda_hyper = wishart.rvs(df = dim + rank, scale = var_W_hyper)
    var_mu_hyper = mvnrnd_pre(var_mu_hyper, (dim + beta0) * var_Lambda_hyper)
    
    idx = list(filter(lambda x: x != k, range(len(factor))))
    var1 = kr_prod(factor[idx[1]], factor[idx[0]]).T
    var2 = kr_prod(var1, var1)
    var3 = (var2 @ ten2mat(tau_ind, k).T).reshape([rank, rank, dim]) + var_Lambda_hyper[:, :, np.newaxis]
    var4 = var1 @ ten2mat(tau_sparse_tensor, k).T + (var_Lambda_hyper @ var_mu_hyper)[:, np.newaxis]
    for i in range(dim):
        factor[k][i, :] = mvnrnd_pre(solve(var3[:, :, i], var4[:, i]), var3[:, :, i])
    return factor[k]

#### - Sampling precision term $\tau$

Draw $\tau\in\text{Gamma}\left(a^{*},b^{*}\right)$ with following parameters:

$$a^{*}=a_0+\frac{1}{2}|\Omega|,~b^{*}=b_0+\frac{1}{2}\sum_{(i,j,t)\in\Omega}\left(y_{ijt}-\sum_{s=1}^{r}u_{is}v_{js}x_{ts}\right)^2.$$


In [9]:
def sample_precision_tau(sparse_tensor, tensor_hat, ind):
    var_alpha = 1e-6 + 0.5 * np.sum(ind)
    var_beta = 1e-6 + 0.5 * np.sum(((sparse_tensor - tensor_hat) ** 2) * ind)
    return np.random.gamma(var_alpha, 1 / var_beta)

### Define Performance Metrics

- **RMSE**
- **MAPE**

In [10]:
def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return  np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

### Define BGCP with `Numpy`

In [11]:
def BGCP(dense_tensor, sparse_tensor, factor, burn_iter, gibbs_iter):
    """Bayesian Gaussian CP (BGCP) decomposition."""
    
    dim = np.array(sparse_tensor.shape)
    rank = factor[0].shape[1]
    if np.isnan(sparse_tensor).any() == False:
        ind = sparse_tensor != 0
        pos_obs = np.where(ind)
        pos_test = np.where((dense_tensor != 0) & (sparse_tensor == 0))
    elif np.isnan(sparse_tensor).any() == True:
        pos_test = np.where((dense_tensor != 0) & (np.isnan(sparse_tensor)))
        ind = ~np.isnan(sparse_tensor)
        pos_obs = np.where(ind)
        sparse_tensor[np.isnan(sparse_tensor)] = 0
    show_iter = 200
    tau = 1
    factor_plus = []
    for k in range(len(dim)):
        factor_plus.append(np.zeros((dim[k], rank)))
    temp_hat = np.zeros(dim)
    tensor_hat_plus = np.zeros(dim)
    for it in range(burn_iter + gibbs_iter):
        tau_ind = tau * ind
        tau_sparse_tensor = tau * sparse_tensor
        for k in range(len(dim)):
            factor[k] = sample_factor(tau_sparse_tensor, tau_ind, factor, k)
        tensor_hat = cp_combine(factor)
        temp_hat += tensor_hat
        tau = sample_precision_tau(sparse_tensor, tensor_hat, ind)
        if it + 1 > burn_iter:
            factor_plus = [factor_plus[k] + factor[k] for k in range(len(dim))]
            tensor_hat_plus += tensor_hat
        if (it + 1) % show_iter == 0 and it < burn_iter:
            temp_hat = temp_hat / show_iter
            print('Iter: {}'.format(it + 1))
            print('MAPE: {:.6}'.format(compute_mape(dense_tensor[pos_test], temp_hat[pos_test])))
            print('RMSE: {:.6}'.format(compute_rmse(dense_tensor[pos_test], temp_hat[pos_test])))
            temp_hat = np.zeros(sparse_tensor.shape)
            print()
    factor = [i / gibbs_iter for i in factor_plus]
    tensor_hat = tensor_hat_plus / gibbs_iter
    print('Imputation MAPE: {:.6}'.format(compute_mape(dense_tensor[pos_test], tensor_hat[pos_test])))
    print('Imputation RMSE: {:.6}'.format(compute_rmse(dense_tensor[pos_test], tensor_hat[pos_test])))
    print()
    
    return tensor_hat, factor

In [12]:
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.3

## Random Missing (RM)
dense_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim2, dim3) + 0.5 - missing_rate)

In [13]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 80
factor = []
for k in range(len(dim)):
    factor.append(0.1 * np.random.randn(dim[k], rank))
burnin_iter = 1000
gibbs_iter = 200
tensor_hat, factor = BGCP(dense_tensor, sparse_tensor, factor, burnin_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 200
MAPE: 0.0844794
RMSE: 3.6278

Iter: 400
MAPE: 0.0839146
RMSE: 3.61087

Iter: 600
MAPE: 0.0836881
RMSE: 3.60329

Iter: 800
MAPE: 0.0835097
RMSE: 3.59796

Iter: 1000
MAPE: 0.0834138
RMSE: 3.59485

Imputation MAPE: 0.083347
Imputation RMSE: 3.59277

Running time: 55.19 minutes


In [14]:
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.7

## Random Missing (RM)
dense_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim2, dim3) + 0.5 - missing_rate)

In [15]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 80
factor = []
for k in range(len(dim)):
    factor.append(0.1 * np.random.randn(dim[k], rank))
burnin_iter = 1000
gibbs_iter = 200
tensor_hat, factor = BGCP(dense_tensor, sparse_tensor, factor, burnin_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 200
MAPE: 0.0860124
RMSE: 3.69509

Iter: 400
MAPE: 0.0857756
RMSE: 3.69618

Iter: 600
MAPE: 0.0855684
RMSE: 3.69159

Iter: 800
MAPE: 0.0854787
RMSE: 3.69049

Iter: 1000
MAPE: 0.0854447
RMSE: 3.68967

Imputation MAPE: 0.0853548
Imputation RMSE: 3.68634

Running time: 56.72 minutes


In [16]:
pos_test = np.where((dense_tensor != 0) & (sparse_tensor == 0))
bgcp_ghat = dense_tensor[pos_test] - tensor_hat[pos_test]
np.save('bgcp_ghat.npy', bgcp_ghat)

In [17]:
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.3

## Non-random Missing (NM)
dense_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim3) + 0.5 - missing_rate)[:, None, :]

In [18]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 10
factor = []
for k in range(len(dim)):
    factor.append(0.1 * np.random.randn(dim[k], rank))
burnin_iter = 1000
gibbs_iter = 200
tensor_hat, factor = BGCP(dense_tensor, sparse_tensor, factor, burnin_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 200
MAPE: 0.105478
RMSE: 4.38065

Iter: 400
MAPE: 0.105073
RMSE: 4.39039

Iter: 600
MAPE: 0.104957
RMSE: 4.38944

Iter: 800
MAPE: 0.10485
RMSE: 4.38626

Iter: 1000
MAPE: 0.10472
RMSE: 4.37648

Imputation MAPE: 0.104695
Imputation RMSE: 4.374

Running time: 2.85 minutes


In [19]:
import numpy as np
import scipy.io
np.random.seed(1000)

missing_rate = 0.7

## Non-random Missing (NM)
dense_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor'].transpose(0, 2, 1)
dim1, dim2, dim3 = dense_tensor.shape
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim3) + 0.5 - missing_rate)[:, None, :]

In [20]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 10
factor = []
for k in range(len(dim)):
    factor.append(0.1 * np.random.randn(dim[k], rank))
burnin_iter = 1000
gibbs_iter = 200
tensor_hat, factor = BGCP(dense_tensor, sparse_tensor, factor, burnin_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 200
MAPE: 0.107123
RMSE: 4.56218

Iter: 400
MAPE: 0.108416
RMSE: 4.72312

Iter: 600
MAPE: 0.108664
RMSE: 4.7655

Iter: 800
MAPE: 0.107668
RMSE: 4.69038

Iter: 1000
MAPE: 0.107409
RMSE: 4.66156

Imputation MAPE: 0.107403
Imputation RMSE: 4.66324

Running time: 2.90 minutes


In [21]:
import numpy as np
np.random.seed(1000)

missing_rate = 0.3

dense_mat = np.load('../datasets/London-data-set/hourly_speed_mat.npy')
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]
dim1, dim2 = dense_mat.shape
del binary_mat

## Random missing (RM)
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)
dense_tensor = dense_mat.reshape([dim1, 30, 24]).transpose(0, 2, 1)
sparse_tensor = sparse_mat.reshape([dim1, 30, 24]).transpose(0, 2, 1)
del dense_mat, sparse_mat

In [22]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 20
factor = []
for k in range(len(dim)):
    factor.append(0.1 * np.random.randn(dim[k], rank))
burnin_iter = 1000
gibbs_iter = 200
tensor_hat, factor = BGCP(dense_tensor, sparse_tensor, factor, burnin_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 200
MAPE: 0.0923704
RMSE: 2.24761

Iter: 400
MAPE: 0.0920785
RMSE: 2.24273

Iter: 600
MAPE: 0.0919728
RMSE: 2.24107

Iter: 800
MAPE: 0.0919203
RMSE: 2.23963

Iter: 1000
MAPE: 0.0918943
RMSE: 2.23878

Imputation MAPE: 0.0918797
Imputation RMSE: 2.23831

Running time: 240.07 minutes


In [23]:
import numpy as np
np.random.seed(1000)

missing_rate = 0.7

dense_mat = np.load('../datasets/London-data-set/hourly_speed_mat.npy')
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]
dim1, dim2 = dense_mat.shape
del binary_mat

## Random missing (RM)
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)
dense_tensor = dense_mat.reshape([dim1, 30, 24]).transpose(0, 2, 1)
sparse_tensor = sparse_mat.reshape([dim1, 30, 24]).transpose(0, 2, 1)
del dense_mat, sparse_mat

In [24]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 20
factor = []
for k in range(len(dim)):
    factor.append(0.1 * np.random.randn(dim[k], rank))
burnin_iter = 1000
gibbs_iter = 200
tensor_hat, factor = BGCP(dense_tensor, sparse_tensor, factor, burnin_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 200
MAPE: 0.0946519
RMSE: 2.30115

Iter: 400
MAPE: 0.0945316
RMSE: 2.29761

Iter: 600
MAPE: 0.0944499
RMSE: 2.29631

Iter: 800
MAPE: 0.0944288
RMSE: 2.29582

Iter: 1000
MAPE: 0.0944131
RMSE: 2.29549

Imputation MAPE: 0.0944034
Imputation RMSE: 2.29526

Running time: 178.31 minutes


In [25]:
pos_test = np.where((dense_tensor != 0) & (sparse_tensor == 0))
bgcp_lhat = dense_tensor[pos_test] - tensor_hat[pos_test]
np.save('bgcp_lhat.npy', bgcp_lhat)

In [26]:
import numpy as np
np.random.seed(1000)

missing_rate = 0.3

dense_mat = np.load('../datasets/London-data-set/hourly_speed_mat.npy')
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]
dim1, dim2 = dense_mat.shape
del binary_mat

## Non-random missing (NM)
dense_tensor = dense_mat.reshape([dim1, 30, 24]).transpose(0, 2, 1)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, 30) + 0.5 - missing_rate)[:, None, :]
del dense_mat

In [27]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 20
factor = []
for k in range(len(dim)):
    factor.append(0.1 * np.random.randn(dim[k], rank))
burnin_iter = 1000
gibbs_iter = 200
tensor_hat, factor = BGCP(dense_tensor, sparse_tensor, factor, burnin_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 200
MAPE: 0.0945315
RMSE: 2.30887

Iter: 400
MAPE: 0.0947113
RMSE: 2.31177

Iter: 600
MAPE: 0.0946104
RMSE: 2.30884

Iter: 800
MAPE: 0.0945902
RMSE: 2.30817

Iter: 1000
MAPE: 0.0945896
RMSE: 2.30836

Imputation MAPE: 0.0945965
Imputation RMSE: 2.30861

Running time: 175.74 minutes


In [28]:
import numpy as np
np.random.seed(1000)

missing_rate = 0.7

dense_mat = np.load('../datasets/London-data-set/hourly_speed_mat.npy')
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]
dim1, dim2 = dense_mat.shape
del binary_mat

## Non-random missing (NM)
dense_tensor = dense_mat.reshape([dim1, 30, 24]).transpose(0, 2, 1)
sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, 30) + 0.5 - missing_rate)[:, None, :]
del dense_mat

In [29]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 20
factor = []
for k in range(len(dim)):
    factor.append(0.1 * np.random.randn(dim[k], rank))
burnin_iter = 1000
gibbs_iter = 200
tensor_hat, factor = BGCP(dense_tensor, sparse_tensor, factor, burnin_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start) / 60.0))

Iter: 200
MAPE: 0.0994217
RMSE: 2.4298

Iter: 400
MAPE: 0.099987
RMSE: 2.44432

Iter: 600
MAPE: 0.0999745
RMSE: 2.44414

Iter: 800
MAPE: 0.0999687
RMSE: 2.4439

Iter: 1000
MAPE: 0.0999433
RMSE: 2.44297

Imputation MAPE: 0.0998856
Imputation RMSE: 2.44155

Running time: 191.90 minutes


### Experiments on PeMS-4W Data Set

In [12]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-4w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 4 * 7]), 0)
random_tensor = np.random.rand(data.values.shape[0], 288, 4 * 7)

missing_rate = 0.3

### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_tensor, binary_tensor

In [13]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 10
factor = []
for k in range(len(dim)):
    factor.append(0.1 * np.random.randn(dim[k], rank))
burnin_iter = 1000
gibbs_iter = 200
BGCP(dense_tensor, sparse_tensor, factor, burnin_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start)/60.0))

Iter: 200
MAPE: 0.0509573
RMSE: 4.34123

Iter: 400
MAPE: 0.050683
RMSE: 4.33272

Iter: 600
MAPE: 0.0505847
RMSE: 4.3285

Iter: 800
MAPE: 0.0504597
RMSE: 4.32072

Iter: 1000
MAPE: 0.0504452
RMSE: 4.31881

Imputation MAPE: 0.0502532
Imputation RMSE: 4.31065

Running time: 287.48 minutes


In [14]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-4w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 4 * 7]), 0)
random_tensor = np.random.rand(data.values.shape[0], 288, 4 * 7)

missing_rate = 0.7

### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_tensor, binary_tensor

In [15]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 10
factor = []
for k in range(len(dim)):
    factor.append(0.1 * np.random.randn(dim[k], rank))
burnin_iter = 1000
gibbs_iter = 200
BGCP(dense_tensor, sparse_tensor, factor, burnin_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start)/60.0))

Iter: 200
MAPE: 0.0508234
RMSE: 4.3172

Iter: 400
MAPE: 0.0502252
RMSE: 4.31527

Iter: 600
MAPE: 0.0502002
RMSE: 4.31466

Iter: 800
MAPE: 0.0501868
RMSE: 4.31439

Iter: 1000
MAPE: 0.050184
RMSE: 4.31425

Imputation MAPE: 0.0501825
Imputation RMSE: 4.31415

Running time: 286.09 minutes


In [16]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-4w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 4 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 4 * 7)

missing_rate = 0.3

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_matrix, binary_tensor

In [17]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 10
factor = []
for k in range(len(dim)):
    factor.append(0.1 * np.random.randn(dim[k], rank))
burnin_iter = 1000
gibbs_iter = 200
BGCP(dense_tensor, sparse_tensor, factor, burnin_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start)/60.0))

Iter: 200
MAPE: 0.0541532
RMSE: 4.57425

Iter: 400
MAPE: 0.0540382
RMSE: 4.5875

Iter: 600
MAPE: 0.0539228
RMSE: 4.58336

Iter: 800
MAPE: 0.0538353
RMSE: 4.58434

Iter: 1000
MAPE: 0.0538185
RMSE: 4.58871

Imputation MAPE: 0.0537746
Imputation RMSE: 4.59074

Running time: 284.77 minutes


In [18]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-4w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 4 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 4 * 7)

missing_rate = 0.7

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_matrix, binary_tensor

In [19]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 10
factor = []
for k in range(len(dim)):
    factor.append(0.1 * np.random.randn(dim[k], rank))
burnin_iter = 1000
gibbs_iter = 200
BGCP(dense_tensor, sparse_tensor, factor, burnin_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start)/60.0))

Iter: 200
MAPE: 0.0599587
RMSE: 5.07417

Iter: 400
MAPE: 0.0609349
RMSE: 5.40117

Iter: 600
MAPE: 0.0606259
RMSE: 5.40451

Iter: 800
MAPE: 0.0608938
RMSE: 5.46919

Iter: 1000
MAPE: 0.0610082
RMSE: 5.49707

Imputation MAPE: 0.0610061
Imputation RMSE: 5.49654

Running time: 285.08 minutes


### Experiments on PeMS-8W Data Set

In [20]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-8w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 8 * 7]), 0)
random_tensor = np.random.rand(data.values.shape[0], 288, 8 * 7)

missing_rate = 0.3

### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_tensor, binary_tensor

In [21]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 10
factor = []
for k in range(len(dim)):
    factor.append(0.1 * np.random.randn(dim[k], rank))
burnin_iter = 1000
gibbs_iter = 200
BGCP(dense_tensor, sparse_tensor, factor, burnin_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start)/60.0))

Iter: 200
MAPE: 0.0536342
RMSE: 4.54811

Iter: 400
MAPE: 0.0528521
RMSE: 4.51873

Iter: 600
MAPE: 0.0528394
RMSE: 4.51789

Iter: 800
MAPE: 0.052841
RMSE: 4.51758

Iter: 1000
MAPE: 0.0528411
RMSE: 4.51738

Imputation MAPE: 0.0528409
Imputation RMSE: 4.51715

Running time: 585.60 minutes


In [22]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-8w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 8 * 7]), 0)
random_tensor = np.random.rand(data.values.shape[0], 288, 8 * 7)

missing_rate = 0.7

### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_tensor, binary_tensor

In [23]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 10
factor = []
for k in range(len(dim)):
    factor.append(0.1 * np.random.randn(dim[k], rank))
burnin_iter = 1000
gibbs_iter = 200
BGCP(dense_tensor, sparse_tensor, factor, burnin_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start)/60.0))

Iter: 200
MAPE: 0.0532732
RMSE: 4.52602

Iter: 400
MAPE: 0.0527793
RMSE: 4.51802

Iter: 600
MAPE: 0.0527547
RMSE: 4.51666

Iter: 800
MAPE: 0.0527249
RMSE: 4.51404

Iter: 1000
MAPE: 0.0527285
RMSE: 4.51299

Imputation MAPE: 0.0527291
Imputation RMSE: 4.5127

Running time: 553.88 minutes


In [24]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-8w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 8 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 8 * 7)

missing_rate = 0.3

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_matrix, binary_tensor

In [25]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 10
factor = []
for k in range(len(dim)):
    factor.append(0.1 * np.random.randn(dim[k], rank))
burnin_iter = 1000
gibbs_iter = 200
BGCP(dense_tensor, sparse_tensor, factor, burnin_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start)/60.0))

Iter: 200
MAPE: 0.0559252
RMSE: 4.69442

Iter: 400
MAPE: 0.0551243
RMSE: 4.67289

Iter: 600
MAPE: 0.0548693
RMSE: 4.66503

Iter: 800
MAPE: 0.0547806
RMSE: 4.66047

Iter: 1000
MAPE: 0.0547388
RMSE: 4.65913

Imputation MAPE: 0.0547163
Imputation RMSE: 4.65863

Running time: 563.42 minutes


In [26]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('../datasets/California-data-set/pems-8w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 8 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 8 * 7)

missing_rate = 0.7

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_matrix, binary_tensor

In [27]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 10
factor = []
for k in range(len(dim)):
    factor.append(0.1 * np.random.randn(dim[k], rank))
burnin_iter = 1000
gibbs_iter = 200
BGCP(dense_tensor, sparse_tensor, factor, burnin_iter, gibbs_iter)
end = time.time()
print('Running time: %.2f minutes'%((end - start)/60.0))

Iter: 200
MAPE: 0.0573149
RMSE: 4.81128

Iter: 400
MAPE: 0.0562119
RMSE: 4.77796

Iter: 600
MAPE: 0.0562737
RMSE: 4.80638

Iter: 800
MAPE: 0.056286
RMSE: 4.81353

Iter: 1000
MAPE: 0.0563006
RMSE: 4.81698

Imputation MAPE: 0.0563073
Imputation RMSE: 4.81831

Running time: 551.31 minutes


### Experiments on London-1M Data Set

In [12]:
import numpy as np
np.random.seed(1000)

missing_rate = 0.3

dense_mat = np.load('../datasets/London-data-set/hourly_speed_mat.npy')
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]

## Random missing (RM)
random_mat = np.random.rand(dense_mat.shape[0], dense_mat.shape[1])
binary_mat = np.round(random_mat + 0.5 - missing_rate)
sparse_mat = np.multiply(dense_mat, binary_mat)

dense_tensor = dense_mat.reshape([dense_mat.shape[0], 30, 24])
sparse_tensor = sparse_mat.reshape([sparse_mat.shape[0], 30, 24])
del dense_mat, sparse_mat

In [13]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 20
factor = []
for k in range(len(dim)):
    factor.append(0.1 * np.random.randn(dim[k], rank))
burn_iter = 1000
gibbs_iter = 200
BGCP(dense_tensor, sparse_tensor, factor, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.0922696
RMSE: 2.2483

Iter: 400
MAPE: 0.0920824
RMSE: 2.24374

Iter: 600
MAPE: 0.092008
RMSE: 2.24153

Iter: 800
MAPE: 0.0919649
RMSE: 2.23992

Iter: 1000
MAPE: 0.0918846
RMSE: 2.23833

Imputation MAPE: 0.0918061
Imputation RMSE: 2.23727

Running time: 12058 seconds


In [14]:
import numpy as np
np.random.seed(1000)

missing_rate = 0.7

dense_mat = np.load('../datasets/London-data-set/hourly_speed_mat.npy')
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]

## Random missing (RM)
random_mat = np.random.rand(dense_mat.shape[0], dense_mat.shape[1])
binary_mat = np.round(random_mat + 0.5 - missing_rate)
sparse_mat = np.multiply(dense_mat, binary_mat)

dense_tensor = dense_mat.reshape([dense_mat.shape[0], 30, 24])
sparse_tensor = sparse_mat.reshape([sparse_mat.shape[0], 30, 24])
del dense_mat, sparse_mat

In [15]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 20
factor = []
for k in range(len(dim)):
    factor.append(0.1 * np.random.randn(dim[k], rank))
burn_iter = 1000
gibbs_iter = 200
BGCP(dense_tensor, sparse_tensor, factor, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.094615
RMSE: 2.3016

Iter: 400
MAPE: 0.0946033
RMSE: 2.30029

Iter: 600
MAPE: 0.0945642
RMSE: 2.29905

Iter: 800
MAPE: 0.0945298
RMSE: 2.29797

Iter: 1000
MAPE: 0.094502
RMSE: 2.29739

Imputation MAPE: 0.0944727
Imputation RMSE: 2.29685

Running time: 11992 seconds


In [16]:
import numpy as np
np.random.seed(1000)

missing_rate = 0.3

dense_mat = np.load('../datasets/London-data-set/hourly_speed_mat.npy')
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]

## Non-random missing (NM)
binary_mat = np.zeros(dense_mat.shape)
random_mat = np.random.rand(dense_mat.shape[0], 30)
for i1 in range(dense_mat.shape[0]):
    for i2 in range(30):
        binary_mat[i1, i2 * 24 : (i2 + 1) * 24] = np.round(random_mat[i1, i2] + 0.5 - missing_rate)
sparse_mat = np.multiply(dense_mat, binary_mat)

dense_tensor = dense_mat.reshape([dense_mat.shape[0], 30, 24])
sparse_tensor = sparse_mat.reshape([sparse_mat.shape[0], 30, 24])
del dense_mat, sparse_mat

In [17]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 20
factor = []
for k in range(len(dim)):
    factor.append(0.1 * np.random.randn(dim[k], rank))
burn_iter = 1000
gibbs_iter = 200
BGCP(dense_tensor, sparse_tensor, factor, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.0944617
RMSE: 2.30578

Iter: 400
MAPE: 0.0946107
RMSE: 2.31102

Iter: 600
MAPE: 0.0946036
RMSE: 2.31093

Iter: 800
MAPE: 0.0945849
RMSE: 2.31061

Iter: 1000
MAPE: 0.0945564
RMSE: 2.31009

Imputation MAPE: 0.0945378
Imputation RMSE: 2.30959

Running time: 11473 seconds


In [18]:
import numpy as np
np.random.seed(1000)

missing_rate = 0.7

dense_mat = np.load('../datasets/London-data-set/hourly_speed_mat.npy')
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]

## Non-random missing (NM)
binary_mat = np.zeros(dense_mat.shape)
random_mat = np.random.rand(dense_mat.shape[0], 30)
for i1 in range(dense_mat.shape[0]):
    for i2 in range(30):
        binary_mat[i1, i2 * 24 : (i2 + 1) * 24] = np.round(random_mat[i1, i2] + 0.5 - missing_rate)
sparse_mat = np.multiply(dense_mat, binary_mat)

dense_tensor = dense_mat.reshape([dense_mat.shape[0], 30, 24])
sparse_tensor = sparse_mat.reshape([sparse_mat.shape[0], 30, 24])
del dense_mat, sparse_mat

In [19]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 20
factor = []
for k in range(len(dim)):
    factor.append(0.1 * np.random.randn(dim[k], rank))
burn_iter = 1000
gibbs_iter = 200
BGCP(dense_tensor, sparse_tensor, factor, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.09921
RMSE: 2.42329

Iter: 400
MAPE: 0.0999265
RMSE: 2.44132

Iter: 600
MAPE: 0.0999539
RMSE: 2.44186

Iter: 800
MAPE: 0.09997
RMSE: 2.44213

Iter: 1000
MAPE: 0.0999865
RMSE: 2.44243

Imputation MAPE: 0.0999842
Imputation RMSE: 2.44213

Running time: 12733 seconds


### Experiments on Guangzhou-2M Data Set

In [20]:
import scipy.io

dense_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor']
random_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/random_tensor.mat')['random_tensor']
missing_rate = 0.3

## Random missing (RM)
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)

In [21]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 80
factor = []
for k in range(len(dim)):
    factor.append(0.1 * np.random.randn(dim[k], rank))
burn_iter = 1000
gibbs_iter = 200
BGCP(dense_tensor, sparse_tensor, factor, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.0839745
RMSE: 3.61647

Iter: 400
MAPE: 0.0835623
RMSE: 3.60507

Iter: 600
MAPE: 0.0833741
RMSE: 3.59895

Iter: 800
MAPE: 0.0832037
RMSE: 3.59308

Iter: 1000
MAPE: 0.083074
RMSE: 3.58745

Imputation MAPE: 0.0829904
Imputation RMSE: 3.58367

Running time: 3108 seconds


In [22]:
import scipy.io

dense_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor']
random_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/random_tensor.mat')['random_tensor']
missing_rate = 0.7

## Random missing (RM)
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)

In [23]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 80
factor = []
for k in range(len(dim)):
    factor.append(0.1 * np.random.randn(dim[k], rank))
burn_iter = 1000
gibbs_iter = 200
BGCP(dense_tensor, sparse_tensor, factor, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.0864222
RMSE: 3.72361

Iter: 400
MAPE: 0.0860259
RMSE: 3.71598

Iter: 600
MAPE: 0.0857265
RMSE: 3.70482

Iter: 800
MAPE: 0.085522
RMSE: 3.69766

Iter: 1000
MAPE: 0.0853367
RMSE: 3.69149

Imputation MAPE: 0.0853178
Imputation RMSE: 3.68971

Running time: 3133 seconds


In [24]:
import scipy.io

dense_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor']
random_matrix = scipy.io.loadmat('../datasets/Guangzhou-data-set/random_matrix.mat')['random_matrix']
missing_rate = 0.3

## Non-random missing (NM)
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[1]):
        binary_tensor[i1, i2, :] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)

In [25]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 10
factor = []
for k in range(len(dim)):
    factor.append(0.1 * np.random.randn(dim[k], rank))
burn_iter = 1000
gibbs_iter = 200
BGCP(dense_tensor, sparse_tensor, factor, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.101898
RMSE: 4.2785

Iter: 400
MAPE: 0.102387
RMSE: 4.3103

Iter: 600
MAPE: 0.102448
RMSE: 4.31487

Iter: 800
MAPE: 0.102476
RMSE: 4.31679

Iter: 1000
MAPE: 0.102471
RMSE: 4.31745

Imputation MAPE: 0.102478
Imputation RMSE: 4.31969

Running time: 159 seconds


In [26]:
import scipy.io

dense_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor']
random_matrix = scipy.io.loadmat('../datasets/Guangzhou-data-set/random_matrix.mat')['random_matrix']
missing_rate = 0.7

## Non-random missing (NM)
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[1]):
        binary_tensor[i1, i2, :] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)

In [28]:
import time
start = time.time()
dim = np.array(sparse_tensor.shape)
rank = 5
factor = []
for k in range(len(dim)):
    factor.append(0.1 * np.random.randn(dim[k], rank))
burn_iter = 1000
gibbs_iter = 200
BGCP(dense_tensor, sparse_tensor, factor, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.113098
RMSE: 4.63985

Iter: 400
MAPE: 0.113084
RMSE: 4.65119

Iter: 600
MAPE: 0.112895
RMSE: 4.64176

Iter: 800
MAPE: 0.112743
RMSE: 4.6363

Iter: 1000
MAPE: 0.112747
RMSE: 4.6376

Imputation MAPE: 0.112771
Imputation RMSE: 4.63918

Running time: 129 seconds


### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>